In [2]:
from lxml import etree, html
import pandas as pd
from datetime import date
import requests

In [3]:
doc = etree.parse('BOE-A-2020-14372.xml')

In [4]:
#etree.tostring(doc,pretty_print=True ,xml_declaration=True, encoding="utf-8")

In [5]:
raiz=doc.getroot()
raiz.tag

'documento'

In [6]:
len(raiz)

4

In [7]:
libro=raiz[0]
libro.tag

'metadatos'

In [8]:
libro=raiz[0]
libro[0].text

'BOE-A-2020-14372'

In [9]:
libro=raiz[1]
libro[0].tag

'{http://www.w3.org/1999/02/22-rdf-syntax-ns#}RDF'

In [10]:
libro=raiz[1]
libro[0].text

'\n            '

In [11]:
Identificador=raiz.find("metadatos/identificador")
Titulo=raiz.find("metadatos/titulo")
URL_pdf=raiz.find("metadatos/url_pdf")

In [12]:
print(Identificador.text)
print(Titulo.text)
print("https://www.boe.es" + URL_pdf.text)


BOE-A-2020-14372
Decreto-ley 35/2020, de 20 de octubre, de modificación del Decreto-ley 19/2020, de 19 de mayo, y del Decreto-ley 29/2020, de 28 de julio, en materia de adopción de medidas sociales y sanitarias para paliar los efectos de la pandemia generada por la COVID-19.
https://www.boe.es/boe/dias/2020/11/18/pdfs/BOE-A-2020-14372.pdf


In [13]:
materia=raiz.find("analisis/materias/materia")
materia.text

'Asistencia sanitaria'

In [14]:
materias=raiz.findall("analisis/materias/materia")
for materia in materias:
    print(materia.text)

Asistencia sanitaria
Asistencia social
Ayudas
Cataluña
Centros residenciales para personas con discapacidad
Epidemias
Establecimientos residenciales para la Tercera Edad
Pagos
Servicios Sociales de las Comunidades Autónomas


In [15]:
type(materias)

list

In [16]:
alertas=raiz.findall("analisis/alertas/alerta")
for alerta in alertas:
    print(alerta.text)

Asuntos sociales
Sanidad


# Recoge resumen diario del BOE de hoy

In [17]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)



Fecha de Hoy = 20201222
dia = 22
mes = 12
año = 2020


In [18]:
URL_XML_resumen =  "https://www.boe.es/diario_boe/xml.php?id=BOE-S-" + str(hoy)

In [19]:
URL_XML_resumen

'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20201222'

In [20]:
#pip install requests for installation



url = URL_XML_resumen
r = requests.get(url)
#r.content
#r.text

In [21]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)
        
#save_html(r.content, './BOEs/resumen.xml')
save_html(r.content, './BOEs/Resumen-BOE-' + hoy + '.xml')

In [22]:
#r.encoding

In [23]:
#resumen = etree.parse('./BOEs/resumen.xml')
resumen = etree.parse('./BOEs/Resumen-BOE-' + hoy + '.xml')

In [24]:
raiz=resumen.getroot()
raiz.tag
raiz_sumario = raiz

In [25]:
seccion = raiz.findall("sumario/diario/seccion")
seccion

[]

In [26]:
for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')
    print(nombre_seccion)

['I. Disposiciones generales']
['II. Autoridades y personal. - A. Nombramientos, situaciones e incidencias']
['II. Autoridades y personal. - B. Oposiciones y concursos']
['III. Otras disposiciones']
['IV. Administración de Justicia']
['V. Anuncios. - A. Contratación del Sector Público']
['V. Anuncios. - B. Otros anuncios oficiales']
['V. Anuncios. - C. Anuncios particulares']
['T.C. Sección del Tribunal Constitucional']


In [27]:
secciones = raiz.findall("sumario/diario/seccion")
for seccion in secciones:
    print(seccion.text)

In [28]:
tabla_resumen = pd.DataFrame()

In [29]:
#for seccion in raiz.xpath('//seccion[contains(@nombre, "III. Otras disposiciones") or contains(@nombre, "I. Disposiciones generales")]'):
for seccion in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales")]'):
#for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')

    for departamento in seccion:
        nombre_departamento = departamento.xpath('@nombre') 

        for epigrafe in departamento:
            nombre_epigrafe = epigrafe.xpath('@nombre')

            for item in epigrafe:
                item_id = item.xpath('@id')
                item_name = item.xpath('.//titulo/text()') 

                item_urlXml = "https://www.boe.es" + str(item.xpath('.//urlXml/text()'))[2:-2]
                #print(item_urlXml)

                tabla_resumen = tabla_resumen.append({'Seccion': nombre_seccion, 
                                                      'Departamento': nombre_departamento, 
                                                      'Epigrafe' : nombre_epigrafe,
                                                      'Item_id': item_id, 
                                                      'Item_Nombre' : item_name, 
                                                      'Item_URL_XML' : item_urlXml},
                                                      ignore_index=True)


In [30]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],[Cuestión de inconstitucionalidad n.º 4701-202...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16670],[I. Disposiciones generales]
1,[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],[Cuestión de inconstitucionalidad n.º 5246-202...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16671],[I. Disposiciones generales]
2,[TRIBUNAL CONSTITUCIONAL],[Conflictos constitucionales],[Conflicto positivo de competencia número 6201...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16672],[I. Disposiciones generales]
3,"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],[Acuerdo Multilateral M332 en virtud de la sec...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16673],[I. Disposiciones generales]
4,[MINISTERIO DE HACIENDA],[Procedimientos tributarios. Gestión informati...,"[Resolución de 15 de diciembre de 2020, del De...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16674],[I. Disposiciones generales]
5,"[MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y M...",[Organización],"[Resolución de 15 de diciembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16675],[I. Disposiciones generales]
6,[COMUNIDAD AUTÓNOMA DE CATALUÑA],[Elecciones al Parlamento de Cataluña. Convoca...,"[Decreto 147/2020, de 21 de diciembre, de diso...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16676],[I. Disposiciones generales]


# Descarga ficheros XML asociados

In [31]:
for item_URL in tabla_resumen['Item_URL_XML']:
    #print(URL)
    r = requests.get(item_URL)
    f = './BOEs/' + item_URL[-16:] + '.xml'
    save_html(r.content, f)

In [32]:

import glob
BOEs = glob.glob('./BOEs/BOE*.xml')
print (BOEs)

['./BOEs/BOE-A-2020-15394.xml', './BOEs/BOE-A-2020-16676.xml', './BOEs/BOE-A-2020-14465.xml', './BOEs/BOE-A-2020-14459.xml', './BOEs/BOE-A-2020-15237.xml', './BOEs/BOE-A-2020-15223.xml', './BOEs/BOE-A-2020-15222.xml', './BOEs/BOE-A-2020-15236.xml', './BOEs/BOE-A-2020-14458.xml', './BOEs/BOE-A-2020-14464.xml', './BOEs/BOE-A-2020-15395.xml', './BOEs/BOE-A-2020-15397.xml', './BOEs/BOE-A-2020-16675.xml', './BOEs/BOE-A-2020-14466.xml', './BOEs/BOE-A-2020-15208.xml', './BOEs/BOE-A-2020-14880.xml', './BOEs/BOE-A-2020-15220.xml', './BOEs/BOE-A-2020-15234.xml', './BOEs/BOE-A-2020-15235.xml', './BOEs/BOE-A-2020-15221.xml', './BOEs/BOE-A-2020-15209.xml', './BOEs/BOE-A-2020-14467.xml', './BOEs/BOE-A-2020-16674.xml', './BOEs/BOE-A-2020-15396.xml', './BOEs/BOE-A-2020-15392.xml', './BOEs/BOE-A-2020-16670.xml', './BOEs/BOE-A-2020-14463.xml', './BOEs/BOE-A-2020-15225.xml', './BOEs/BOE-A-2020-15231.xml', './BOEs/BOE-A-2020-15219.xml', './BOEs/BOE-A-2020-15218.xml', './BOEs/BOE-A-2020-15230.xml', './BOEs

In [33]:
#import os
#path = './BOEs'
#BOEs = [f for f in os.listdir(path) if f.startswith('BOE')]
#BOEs

In [34]:
tabla_materias = pd.DataFrame()

for BOE in BOEs:
    #print (BOE)
    BOE_XML = etree.parse(BOE)
    raiz=BOE_XML.getroot()

    materias = [materia.text for materia in raiz.findall('analisis/materias/materia')]
    alertas = [alerta.text for alerta in raiz.findall('analisis/alertas/alerta')]
    #Item_Id = '[' + BOE[7:23] + ']'
    #Item_Id = raiz.find('metadatos/identificador')
    Item_Id = raiz.xpath(".//identificador/text()")
 
    #print(Item_Id, materias, alertas)
    #tabla_materias.loc[['Item_id'], ['Materias']] = materias
    tabla_materias = tabla_materias.append({'Item_id': Item_Id,
                                            'Materias' : materias,
                                            'Alertas' : alertas},
                                            ignore_index=True)

    #tabla_materias.Item_id.apply(list)
    #fila_tabla = tabla_resumen.set_index('Item_id').filter(like=BOE[18:23], axis=0).reset_index()
    #print(fila_tabla['Item_id'])

    #tabla_resumen.loc[fila_tabla['Item_id']]['Materias'] = materias


In [35]:
tabla_materias.sort_values('Item_id')

,Alertas,Item_id,Materias
154,"[Relaciones internacionales, Sanidad, Sistema ...",[BOE-A-2020-14457],"[Acuerdos internacionales, Créditos Presupuest..."
8,"[Educación y enseñanza, Relaciones internacion...",[BOE-A-2020-14458],"[Acuerdos internacionales, Barcelona, Centros ..."
3,"[Educación y enseñanza, Relaciones internacion...",[BOE-A-2020-14459],"[Acuerdos internacionales, Barcelona, Centros ..."
38,"[Administración de Justicia, Sistema financiero]",[BOE-A-2020-14460],"[Ayuntamientos, Créditos Presupuestarios, Juzg..."
45,[Sistema financiero],[BOE-A-2020-14461],[Administración y Contabilidad de la Hacienda ...
...,...,...,...
46,"[Derecho Constitucional, Organización de la Ad...",[BOE-A-2020-16672],"[Canarias, Conflictos de Competencia, Epidemia..."
37,"[Relaciones internacionales, Transportes y trá...",[BOE-A-2020-16673],"[Acuerdos internacionales, Análisis, Energía n..."
22,"[Consumidores y usuarios, Sistema tributario, ...",[BOE-A-2020-16674],[Agencia Estatal de la Administración Tributar...
12,"[Derecho Administrativo, Seguridad Social]",[BOE-A-2020-16675],"[Alicante, Apremios, Gipuzkoa, Madrid, Procedi..."


In [36]:
print(raiz.tag)

documento


In [37]:
for materias in raiz.xpath('.//materias'):
    materia = seccion.xpath('.//materia/text()')
    print(materia)

[]


In [38]:
materias=raiz.findall("analisis/materias/materia")
for materia in materias:
    print(materia.text)

Asociaciones
Fundaciones
Medio ambiente
Ministerio para la Transición Ecológica y el Reto Demográfico
Subvenciones


In [39]:
alertas=raiz.findall("analisis/alertas/alerta")
for alerta in alertas:
    print(alerta.text)

Medio ambiente


In [40]:
#tabla_resumen.loc[tabla_resumen['Item_id'] == '[' + BOE[7:23] + ']']
print(tabla_resumen.loc[tabla_resumen['Item_id'] == '[BOE-A-2020-14457]'])

Empty DataFrame
Columns: [Departamento, Epigrafe, Item_Nombre, Item_URL_XML, Item_id, Seccion]
Index: []


In [41]:
BOE[7:23]

'BOE-A-2020-15399'

In [42]:
tabla_resumen['Item_id']

0    [BOE-A-2020-16670]
1    [BOE-A-2020-16671]
2    [BOE-A-2020-16672]
3    [BOE-A-2020-16673]
4    [BOE-A-2020-16674]
5    [BOE-A-2020-16675]
6    [BOE-A-2020-16676]
Name: Item_id, dtype: object

In [43]:
tabla_materias['Item_id']

0      [BOE-A-2020-15394]
1      [BOE-A-2020-16676]
2      [BOE-A-2020-14465]
3      [BOE-A-2020-14459]
4      [BOE-A-2020-15237]
              ...        
159    [BOE-A-2020-15207]
160    [BOE-A-2020-15206]
161    [BOE-A-2020-15212]
162    [BOE-A-2020-15400]
163    [BOE-A-2020-15399]
Name: Item_id, Length: 164, dtype: object

In [44]:
tabla_resumen.dtypes

Departamento    object
Epigrafe        object
Item_Nombre     object
Item_URL_XML    object
Item_id         object
Seccion         object
dtype: object

In [45]:
tabla_resumen.set_index('Item_id').filter(like='14875', axis=0)

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Seccion
Item_id,,,,,


In [46]:
#tabla_resumen = tabla_resumen.astype(str)
#tabla_resumen['Joined'] = tabla_resumen.Item_id.apply(', '.join)
#tabla_resumen['Item_id']=tabla_resumen['Item_id']

In [47]:
tabla_resumen.dtypes

Departamento    object
Epigrafe        object
Item_Nombre     object
Item_URL_XML    object
Item_id         object
Seccion         object
dtype: object

In [48]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],[Cuestión de inconstitucionalidad n.º 4701-202...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16670],[I. Disposiciones generales]
1,[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],[Cuestión de inconstitucionalidad n.º 5246-202...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16671],[I. Disposiciones generales]
2,[TRIBUNAL CONSTITUCIONAL],[Conflictos constitucionales],[Conflicto positivo de competencia número 6201...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16672],[I. Disposiciones generales]
3,"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],[Acuerdo Multilateral M332 en virtud de la sec...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16673],[I. Disposiciones generales]
4,[MINISTERIO DE HACIENDA],[Procedimientos tributarios. Gestión informati...,"[Resolución de 15 de diciembre de 2020, del De...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16674],[I. Disposiciones generales]
5,"[MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y M...",[Organización],"[Resolución de 15 de diciembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16675],[I. Disposiciones generales]
6,[COMUNIDAD AUTÓNOMA DE CATALUÑA],[Elecciones al Parlamento de Cataluña. Convoca...,"[Decreto 147/2020, de 21 de diciembre, de diso...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16676],[I. Disposiciones generales]


In [49]:
#for row in tabla_resumen:
#    tabla_resumen.loc[row['Item_id']]['Materias'] = tabla_materias.loc[row['Item_id']]['Materias']

In [50]:
tabla_resumen['Item_id'] = tabla_resumen['Item_id'].astype(str)
tabla_materias['Item_id'] = tabla_materias['Item_id'].astype(str)
tabla_resultados = tabla_resumen.set_index('Item_id').join(tabla_materias.set_index('Item_id'))


In [51]:
tabla_resultados

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Seccion,Alertas,Materias
Item_id,,,,,,,
['BOE-A-2020-16670'],[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],[Cuestión de inconstitucionalidad n.º 4701-202...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Civil, Derecho Constitucional]","[Código Civil, Cuestiones de inconstitucionali..."
['BOE-A-2020-16671'],[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],[Cuestión de inconstitucionalidad n.º 5246-202...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Administrativo, Derecho Constituciona...","[Cuestiones de inconstitucionalidad, Delitos e..."
['BOE-A-2020-16672'],[TRIBUNAL CONSTITUCIONAL],[Conflictos constitucionales],[Conflicto positivo de competencia número 6201...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Constitucional, Organización de la Ad...","[Canarias, Conflictos de Competencia, Epidemia..."
['BOE-A-2020-16673'],"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Acuerdos internacionales administrativos],[Acuerdo Multilateral M332 en virtud de la sec...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Relaciones internacionales, Transportes y trá...","[Acuerdos internacionales, Análisis, Energía n..."
['BOE-A-2020-16674'],[MINISTERIO DE HACIENDA],[Procedimientos tributarios. Gestión informati...,"[Resolución de 15 de diciembre de 2020, del De...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Consumidores y usuarios, Sistema tributario, ...",[Agencia Estatal de la Administración Tributar...
['BOE-A-2020-16675'],"[MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y M...",[Organización],"[Resolución de 15 de diciembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Administrativo, Seguridad Social]","[Alicante, Apremios, Gipuzkoa, Madrid, Procedi..."
['BOE-A-2020-16676'],[COMUNIDAD AUTÓNOMA DE CATALUÑA],[Elecciones al Parlamento de Cataluña. Convoca...,"[Decreto 147/2020, de 21 de diciembre, de diso...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Constitucional, Organización de la Ad...","[Cataluña, Diputados, Elecciones autonómicas, ..."


In [52]:
#tabla_resultados.to_csv('./BOEs/Resultados.csv', index=False)
tabla_resultados.to_csv('./BOEs/Resultados-BOE-' + hoy + '.csv', index=False)

## Genera Split de ALertas y crea una fila para cada una

### Los items sin Alerta se desechan

In [53]:
tabla_resultados.Alertas.apply(pd.Series) \
    .merge(tabla_resultados, left_index = True, right_index = True) \
    .drop(["Alertas"], axis = 1) \
    .reset_index() \
    .melt(id_vars = ['Item_id','Item_Nombre','Seccion','Departamento','Epigrafe','Item_URL_XML','Materias',], value_name = "Alerta") \
    .drop("variable", axis = 1) \
    .dropna() \
    .sort_values('Alerta')

,Item_id,Item_Nombre,Seccion,Departamento,Epigrafe,Item_URL_XML,Materias,Alerta
4,['BOE-A-2020-16674'],"[Resolución de 15 de diciembre de 2020, del De...",[I. Disposiciones generales],[MINISTERIO DE HACIENDA],[Procedimientos tributarios. Gestión informati...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[Agencia Estatal de la Administración Tributar...,Consumidores y usuarios
1,['BOE-A-2020-16671'],[Cuestión de inconstitucionalidad n.º 5246-202...,[I. Disposiciones generales],[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Cuestiones de inconstitucionalidad, Delitos e...",Derecho Administrativo
5,['BOE-A-2020-16675'],"[Resolución de 15 de diciembre de 2020, de la ...",[I. Disposiciones generales],"[MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y M...",[Organización],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Alicante, Apremios, Gipuzkoa, Madrid, Procedi...",Derecho Administrativo
0,['BOE-A-2020-16670'],[Cuestión de inconstitucionalidad n.º 4701-202...,[I. Disposiciones generales],[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Código Civil, Cuestiones de inconstitucionali...",Derecho Civil
7,['BOE-A-2020-16670'],[Cuestión de inconstitucionalidad n.º 4701-202...,[I. Disposiciones generales],[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Código Civil, Cuestiones de inconstitucionali...",Derecho Constitucional
6,['BOE-A-2020-16676'],"[Decreto 147/2020, de 21 de diciembre, de diso...",[I. Disposiciones generales],[COMUNIDAD AUTÓNOMA DE CATALUÑA],[Elecciones al Parlamento de Cataluña. Convoca...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Cataluña, Diputados, Elecciones autonómicas, ...",Derecho Constitucional
8,['BOE-A-2020-16671'],[Cuestión de inconstitucionalidad n.º 5246-202...,[I. Disposiciones generales],[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Cuestiones de inconstitucionalidad, Delitos e...",Derecho Constitucional
2,['BOE-A-2020-16672'],[Conflicto positivo de competencia número 6201...,[I. Disposiciones generales],[TRIBUNAL CONSTITUCIONAL],[Conflictos constitucionales],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Canarias, Conflictos de Competencia, Epidemia...",Derecho Constitucional
15,['BOE-A-2020-16671'],[Cuestión de inconstitucionalidad n.º 5246-202...,[I. Disposiciones generales],[TRIBUNAL CONSTITUCIONAL],[Cuestiones de inconstitucionalidad],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Cuestiones de inconstitucionalidad, Delitos e...",Derecho Penal
9,['BOE-A-2020-16672'],[Conflicto positivo de competencia número 6201...,[I. Disposiciones generales],[TRIBUNAL CONSTITUCIONAL],[Conflictos constitucionales],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Canarias, Conflictos de Competencia, Epidemia...",Organización de la Administración


In [54]:
tabla_resultados['Epigrafe'].str.join(',')

Item_id
['BOE-A-2020-16670']                   Cuestiones de inconstitucionalidad
['BOE-A-2020-16671']                   Cuestiones de inconstitucionalidad
['BOE-A-2020-16672']                          Conflictos constitucionales
['BOE-A-2020-16673']             Acuerdos internacionales administrativos
['BOE-A-2020-16674']    Procedimientos tributarios. Gestión informatizada
['BOE-A-2020-16675']                                         Organización
['BOE-A-2020-16676']    Elecciones al Parlamento de Cataluña. Convocat...
Name: Epigrafe, dtype: object

In [55]:
from pprint import pprint

## Convierte estructura XML en diccionario

In [56]:
BOE_XML = etree.parse('./BOEs/BOE-A-2020-14457.xml')
raiz=BOE_XML.getroot()

In [57]:
d = [{x.tag: x.text for x in c} for c in raiz]
pprint(d)

[{'departamento': 'Ministerio de Asuntos Exteriores, Unión Europea y '
                  'Cooperación',
  'diario': 'Boletín Oficial del Estado',
  'diario_numero': '304',
  'estado_consolidacion': None,
  'estatus_derogacion': 'N',
  'estatus_legislativo': 'L',
  'fecha_derogacion': None,
  'fecha_disposicion': '20201030',
  'fecha_publicacion': '20201119',
  'fecha_vigencia': '20201101',
  'identificador': 'BOE-A-2020-14457',
  'judicialmente_anulada': 'N',
  'letra_imagen': 'A',
  'numero_oficial': None,
  'origen_legislativo': 'Estatal',
  'pagina_final': '101023',
  'pagina_inicial': '101017',
  'rango': 'Acuerdo Internacional',
  'seccion': '1',
  'subseccion': None,
  'suplemento_letra_imagen': None,
  'suplemento_pagina_final': None,
  'suplemento_pagina_inicial': None,
  'titulo': 'Acuerdo Internacional Administrativo, para el periodo 2020-2021, '
            'entre el Ministerio de Sanidad del Reino de España y la '
            'Organización Mundial de la Salud, para llevar a

In [58]:
pprint(raiz[2].tag)

'texto'


In [59]:
parrafos=raiz.findall("texto/p")
for parrafo in parrafos:
    print(parrafo.text)

ACUERDO INTERNACIONAL ADMINISTRATIVO, PARA EL PERÍODO 2020-2021, ENTRE EL MINISTERIO DE SANIDAD DEL REINO DE ESPAÑA Y LA ORGANIZACIÓN MUNDIAL DE LA SALUD, PARA LLEVAR A CABO ACTIVIDADES DE COOPERACIÓN EN MATERIA DE DONACIÓN Y TRASPLANTE DE ÓRGANOS, CÉLULAS Y TEJIDOS HUMANOS AL AMPARO DEL ACUERDO MARCO DE COOPERACIÓN ENTRE EL REINO DE ESPAÑA Y LA ORGANIZACIÓN MUNDIAL DE LA SALUD, DE 12 DE SEPTIEMBRE DE 2001
Signatarios:
Salvador Illa Roca, Ministro de Sanidad del Reino de España, en virtud del Real Decreto 8/2020, de 12 de enero, por el que se dispone su nombramiento, y en virtud de las facultades otorgadas en el artículo 48.2 de la Ley 40/2015, de 1 de octubre, de Régimen Jurídico del Sector Público.
Mariângela Simão, Subdirectora General de Acceso a Medicamentos, Vacunas y Productos Farmacéuticos de la Organización Mundial de la Salud (en adelante, la OMS), en representación de dicha Organización.
Los signatarios suscriben el presente documento en el marco jurídico del Acuerdo Marco d

# DOGC

In [60]:
URL_HTML_resumen =  "https://dogc.gencat.cat/es/index.html?newLang=es_ES&language=es_ES"

In [61]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get(URL_HTML_resumen)

In [62]:
r.html.links

{'/ca/index.html?newLang=es_ES&language=es_ES',
 '/ca/pdogc_serveis/cerca-republica/',
 '/ca/pdogc_serveis/pdogc_dogc_mobi/',
 '/es/ajuda/mapaweb/',
 '/es/index.html?newLang=es_ES&language=es_ES',
 '/es/pdogc_base_de_dades_del_dogc/',
 '/es/pdogc_canals_interns/pdogc_cercador_de_normativa/?destParam=cercaAdv',
 '/es/pdogc_canals_interns/pdogc_que_es_el_codi_de_verificacio/',
 '/es/pdogc_eadop/',
 '/es/pdogc_enllacos/',
 '/es/pdogc_preguntes_frequents/',
 '/es/pdogc_serveis/',
 '/es/pdogc_serveis/pdogc_normes_urbanistiques_1985-2002/',
 '/es/pdogc_serveis/pdogc_serveis_de_informacio/',
 '/es/pdogc_serveis/pdogc_serveis_de_informacio/pdogc_alertes_dogc/',
 '/es/pdogc_serveis/pdogc_serveis_de_publicacio_al_dogc/Publicacio_Anuncis_situacio_CoV-2/',
 '/es/pdogc_serveis/pdogc_serveis_de_publicacio_al_dogc/pdogc_dogc_eacat/index.html',
 '/es/pdogc_sobre_el_dogc/',
 'http://bop.diba.es/',
 'http://llibreria.gencat.cat/index.php?language=es',
 'http://web.gencat.cat/es/temes/',
 'http://www.boe

In [63]:
r.html.xpath('//*[@id="titol"]/span/text()')

[' DOGC núm.\r\n\t\t\t\t\t 8301 (22/12/2020)']

In [64]:
r.html.xpath('//*[@id="titol"]/h3/text()')


['\r\n\t\t\t\t\tÚltimo DOGC publicado\r\n\t\t\t\t']

In [65]:
r.html.lxml.tag

'html'

In [66]:
r.html.lxml.xpath('//*[@id="titol"]/span')

[<Element span at 0x7fa2d0438cc0>]

In [67]:
#import requests
#from lxml import html

response = requests.get(URL_HTML_resumen)

tree = html.fromstring(response.text)

# fetches all useful elements 
useful = tree.xpath('//*[@id="titol"]/span/text()')

print(useful)


[' DOGC núm.\r\n\t\t\t\t\t 8301 (22/12/2020)']


In [68]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

['/es/pdogc_canals_interns/pdogc_sumari_del_dogc/']

In [69]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@name')

['anexos']

In [70]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@value')

['1']

In [71]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form')

[<Element form at 0x7fa2d2e3a400>]

In [72]:
useful = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/input[3]/@value')

print(useful)

['8301']


In [73]:
tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

['/es/pdogc_canals_interns/pdogc_sumari_del_dogc/']

In [74]:
cadena = ''
for input in tree.xpath('//*[@id="sumari"]/ul/li[1]/form'):
    argumentos = input.xpath('./input/@name')
    valores = input.xpath('./input/@value')
    print(argumentos, valores)

for indice in range(len(argumentos)):
    cadena += argumentos[indice] + '=' + valores[indice] + '&'

argumentos[1]
cadena[:-1]

['anexos', 'language', 'numDOGC', 'seccion'] ['1', 'es_ES', '8301', '0']


'anexos=1&language=es_ES&numDOGC=8301&seccion=0'

## Recoge Nombre Secciones Sumario

In [75]:
import re
seccion = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/a/text()')
seccion = re.sub('(\\r|\\n|\\t|\ )+', '', seccion[0])
seccion

'DISPOSICIONES'

## Recoge Nombre Secciones Sumario

In [76]:
secciones = tree.xpath('//*[@id="sumari"]/ul/li')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion = seccion.xpath('./form/a/text()')
    seccion = re.sub('(\\r|\\n|\\t)+', '', seccion[0])
    print(seccion[1:-1])
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion[1:-1]}, ignore_index=True)


   DISPOSICIONES 
   CARGOS Y PERSONAL
   ANUNCIOS DE LA GENERALIDAD DE CATALUÑA
   ANUNCIOS DE LA ADMINISTRACIÓN LOCAL
   ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA 
   ANUNCIOS VARIOS


In [77]:
df_secciones_sumarios

,Seccion
0,DISPOSICIONES
1,CARGOS Y PERSONAL
2,ANUNCIOS DE LA GENERALIDAD DE CATALUÑA
3,ANUNCIOS DE LA ADMINISTRACIÓN LOCAL
4,ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA
5,ANUNCIOS VARIOS


## Recoge Valores para formar URLs Secciones Sumario

In [78]:
URL_base_sumario = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

df_URL_sumarios = pd.DataFrame()
for seccion in secciones:    
    cadena = ''
    for input in seccion.xpath('./form'):
        argumentos = input.xpath('./input/@name')
        valores = input.xpath('./input/@value')
        #print(argumentos, valores)

    for indice in range(len(argumentos)):
        cadena += argumentos[indice] + '=' + valores[indice] + '&'

    URL_sumario = 'https://dogc.gencat.cat' + str(URL_base_sumario[0]) + '?' + str(cadena[:-1])
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8301&seccion=0
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8301&seccion=1
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8301&seccion=2
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8301&seccion=3
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8301&seccion=4
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8301&seccion=5


In [79]:
df_URL_sumarios

,URL_Seccion
0,https://dogc.gencat.cat/es/pdogc_canals_intern...
1,https://dogc.gencat.cat/es/pdogc_canals_intern...
2,https://dogc.gencat.cat/es/pdogc_canals_intern...
3,https://dogc.gencat.cat/es/pdogc_canals_intern...
4,https://dogc.gencat.cat/es/pdogc_canals_intern...
5,https://dogc.gencat.cat/es/pdogc_canals_intern...


In [80]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [81]:
df_sumarios

,Seccion,URL_Seccion
0,DISPOSICIONES,https://dogc.gencat.cat/es/pdogc_canals_intern...
1,CARGOS Y PERSONAL,https://dogc.gencat.cat/es/pdogc_canals_intern...
2,ANUNCIOS DE LA GENERALIDAD DE CATALUÑA,https://dogc.gencat.cat/es/pdogc_canals_intern...
3,ANUNCIOS DE LA ADMINISTRACIÓN LOCAL,https://dogc.gencat.cat/es/pdogc_canals_intern...
4,ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA,https://dogc.gencat.cat/es/pdogc_canals_intern...
5,ANUNCIOS VARIOS,https://dogc.gencat.cat/es/pdogc_canals_intern...


In [82]:
df_sumarios['URL_Seccion'][4]

'https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8301&seccion=4'

## Recoge Items en Seccion Disposiciones

In [83]:
### Recoge Items en Seccion Disposiciones

response = requests.get(df_sumarios['URL_Seccion'][0])
sumario_HTML = html.fromstring(response.text)

#seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div[1]/text()')
seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div/text()')
bloques = sumario_HTML.xpath('//*[@id="disposicions"]')
df_disposiciones = pd.DataFrame(columns=['Seccion','Item_Name','PDF_link'])
for bloque in bloques: 
    item_name = bloque.xpath('./div/p/text()')
    #item_name = re.sub('(\\r|\\n|\\t)+', '', item_name[0])
    pdf_link = bloque.xpath('./div/div/a[4]/@href')

for row in range(len(pdf_link)):
    df_disposiciones = df_disposiciones.append({'Item_Name': re.sub('(\\r|\\n|\\t)+', '',                                                                               item_name[row]),
                                            'PDF_link' : pdf_link[row],
                                            'Seccion' : seccion[0][:-1]},
                                            ignore_index=True)

df_disposiciones

,Seccion,Item_Name,PDF_link
0,DISPOSICIONES,...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
1,DISPOSICIONES,"DECRETO 147/2020, de 21 de diciembre, de disol...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
2,DISPOSICIONES,"ORDEN VEH/220/2020, de 17 de diciembre, sobre ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
3,DISPOSICIONES,"ORDEN TES/224/2020, de 17 de diciembre, por la...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
4,DISPOSICIONES,"RESOLUCIÓN TES/3341/2020, de 17 de diciembre, ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
5,DISPOSICIONES,"RESOLUCIÓN TES/3381/2020, de 21 de diciembre, ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
6,DISPOSICIONES,"EDICTO de 9 de diciembre de 2020, sobre acuerd...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
7,DISPOSICIONES,...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
8,DISPOSICIONES,"RESOLUCIÓN CLT/3342/2020, de 15 de diciembre, ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
9,DISPOSICIONES,...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...


In [84]:
pdf_link

['https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889491&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889516&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889505&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889502&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889494&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889473&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889510&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889484&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889534&type=01&